In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-10-20 15:45 ajna         INFO     Configuração de log efetuada


2025-10-20 15:45:03,698 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Analisar informações do Mercante

In [4]:
TIPO_TRAFEGO_EXPORTACAO = 7
TIPO_TRAFEGO_IMPORTACAO = 5
# ultimos_3_meses = 

In [5]:
sql_mercante_exportacao = '''select i.codigoConteiner, c.codigoTerminalCarregamento, c.portoOrigemCarga, c.portoDestFinal, 
c.paisDestinoFinalMercante, m.portoCarregamento, m.portoDescarregamento
from conhecimentosresumo c
inner join manifestosresumo m on c.manifestoCE = m.numero
inner join itensresumo i on i.numeroCEmercante = c.numeroCEmercante
where c.dataEmissao between "2025-06-01" and "2025-10-01"
and c.tipoTrafego = 7 and i.codigoConteiner !="" and portoOrigemCarga = "BRSSZ"
'''
df_mercante_exportacao = pd.read_sql(sql_mercante_exportacao, engine)

In [6]:
df_mercante_exportacao

,codigoConteiner,codigoTerminalCarregamento,portoOrigemCarga,portoDestFinal,paisDestinoFinalMercante,portoCarregamento,portoDescarregamento
0,FCIU5666469,BRSSZ009,BRSSZ,JPMIZ,JP,BRSSZ,KRPUS
1,NYKU3617382,BRSSZ009,BRSSZ,JPMIZ,JP,BRSSZ,KRPUS
2,NYKU9779457,BRSSZ009,BRSSZ,JPMIZ,JP,BRSSZ,KRPUS
3,TCKU2348423,BRSSZ009,BRSSZ,JPMIZ,JP,BRSSZ,KRPUS
4,TCLU3061646,BRSSZ009,BRSSZ,JPMIZ,JP,BRSSZ,KRPUS
...,...,...,...,...,...,...,...
281806,HLBU1686944,BRSSZ016,BRSSZ,CMDLA,CM,BRSSZ,BRITJ
281807,HLBU1856336,BRSSZ016,BRSSZ,CMDLA,CM,BRSSZ,BRITJ
281808,HLBU1891511,BRSSZ016,BRSSZ,CMDLA,CM,BRSSZ,BRITJ
281809,TCLU9303270,BRSSZ016,BRSSZ,CMDLA,CM,BRSSZ,BRITJ


In [7]:
sql_mercante_exportacao_agrupado = '''select c.codigoTerminalCarregamento, c.portoDestFinal, c.paisDestinoFinalMercante,
m.portoDescarregamento, count(i.codigoConteiner) as conteineres
from conhecimentosresumo c
inner join manifestosresumo m on c.manifestoCE = m.numero
inner join itensresumo i on i.numeroCEmercante = c.numeroCEmercante
where c.dataEmissao between "2025-06-01" and "2025-10-01"
and c.tipoTrafego = 7 and i.codigoConteiner !="" and portoOrigemCarga = "BRSSZ"
group by c.codigoTerminalCarregamento, c.portoDestFinal, c.paisDestinoFinalMercante, m.portoDescarregamento'''
df_mercante_exportacao_agrupado = pd.read_sql(sql_mercante_exportacao_agrupado, engine)

In [8]:
df_mercante_exportacao_agrupado

,codigoTerminalCarregamento,portoDestFinal,paisDestinoFinalMercante,portoDescarregamento,conteineres
0,BRIGI001,LBBEY,LB,MAPTM,1
1,BRIOA001,IDJKT,ID,SGSIN,15
2,BRPNG002,CNSHA,CN,CNSHA,1
3,BRPNG002,GHTEM,GH,ZADUR,3
4,BRPNG002,ZADUR,ZA,ZADUR,1
...,...,...,...,...,...
2487,BRSSZ058,ZAPLZ,ZA,ESALG,99
2488,BRSSZ058,ZAPLZ,ZA,MAPTM,18
2489,BRSSZ058,ZAPLZ,ZA,NAWVB,30
2490,BRSSZ058,ZAZBA,ZA,ESLPA,133


In [9]:
df_mercante_exportacao_agrupado.groupby('codigoTerminalCarregamento').conteineres.sum().sort_values(ascending=False)

codigoTerminalCarregamento
BRSSZ058    133732
BRSSZ016     95066
BRSSZ009     50161
BRSSZ057      2827
BRIOA001        15
BRPNG002         5
BRRIO002         2
BRSSZ031         2
BRIGI001         1
Name: conteineres, dtype: int64

In [10]:
df_mercante_exportacao.codigoTerminalCarregamento.value_counts()

codigoTerminalCarregamento
BRSSZ058    133732
BRSSZ016     95066
BRSSZ009     50161
BRSSZ057      2827
BRIOA001        15
BRPNG002         5
BRSSZ031         2
BRRIO002         2
BRIGI001         1
Name: count, dtype: int64

In [11]:
ordenado_pais = (
    df_mercante_exportacao_agrupado.groupby('paisDestinoFinalMercante')['conteineres']
      .sum()
      .reset_index()
      .sort_values(by='conteineres', ascending=False)
)
ordenado_pais[:10]

,paisDestinoFinalMercante,conteineres
27,CN,41234
149,US,25208
70,IN,12601
101,MX,11550
108,NL,9482
28,CO,8130
35,DE,6234
76,JP,6112
113,PE,6038
5,AR,6035


In [12]:
df_mercante_exportacao.paisDestinoFinalMercante.value_counts()[:10]

paisDestinoFinalMercante
CN    41234
US    25208
IN    12601
MX    11550
NL     9482
CO     8130
DE     6234
JP     6112
PE     6038
AR     6035
Name: count, dtype: int64

In [13]:
top5_por_terminal = (
    df_mercante_exportacao.groupby('codigoTerminalCarregamento')['paisDestinoFinalMercante']
    .value_counts()
    .groupby(level=0)
    .head(10)
)

top5_por_terminal

codigoTerminalCarregamento  paisDestinoFinalMercante
BRIGI001                    LB                              1
BRIOA001                    ID                             15
BRPNG002                    GH                              3
                            CN                              1
                            ZA                              1
BRRIO002                    CO                              1
                            US                              1
BRSSZ009                    US                          12064
                            CN                          11852
                            MX                           2912
                            CO                           2592
                            AR                           2237
                            KR                           1602
                            VN                           1601
                            ZA                           1497
                 

In [14]:
df_risco_paises = pd.read_sql('select * from risco_paises', engine)
df_risco_paises

,sigla,nome,escaneamento
0,AE,Emirados Árabes Unidos,1
1,AG,Antígua e Barbuda,0
2,AL,Albânia,1
3,AN,Antilhas Holandesas,0
4,AO,Angola,1
...,...,...,...
143,VE,Venezuela,0
144,VI,Ilhas Virgens (U.S.),0
145,VN,Vietnã,0
146,YE,Iêmen,0


In [15]:
df_mercante_exportacao['sigla_dest_terminal'] = df_mercante_exportacao['portoDestFinal'].str[:2]
df_mercante_exportacao['sigla_pais_final'] = df_mercante_exportacao['paisDestinoFinalMercante'].str[:2]
df_mercante_exportacao['sigla_porto_descar'] = df_mercante_exportacao['portoDescarregamento'].str[:2]


In [16]:
df_mercante_exportacao_agrupado['sigla_dest_terminal'] = df_mercante_exportacao_agrupado['portoDestFinal'].str[:2]
df_mercante_exportacao_agrupado['sigla_pais_final'] = df_mercante_exportacao_agrupado['paisDestinoFinalMercante'].str[:2]
df_mercante_exportacao_agrupado['sigla_porto_descar'] = df_mercante_exportacao_agrupado['portoDescarregamento'].str[:2]

In [17]:
siglas_risco1 = df_risco_paises[df_risco_paises['escaneamento'] == 1]['sigla'].unique()

In [18]:
mask = (
    df_mercante_exportacao['sigla_dest_terminal'].isin(siglas_risco1)
    | df_mercante_exportacao['sigla_pais_final'].isin(siglas_risco1)
    | df_mercante_exportacao['sigla_porto_descar'].isin(siglas_risco1)
)
df_filtrado = df_mercante_exportacao[mask]
print(len(df_filtrado), len(df_mercante_exportacao))

105859 281811


In [19]:
mask = (
    df_mercante_exportacao_agrupado['sigla_dest_terminal'].isin(siglas_risco1)
    | df_mercante_exportacao_agrupado['sigla_pais_final'].isin(siglas_risco1)
    | df_mercante_exportacao_agrupado['sigla_porto_descar'].isin(siglas_risco1)
)
df_filtrado_agrupado = df_mercante_exportacao_agrupado[mask]

In [20]:
df_exportacao_agrupado = (
    df_mercante_exportacao_agrupado.groupby('codigoTerminalCarregamento')['conteineres']
      .sum()
      .reset_index()
      .sort_values(by='conteineres', ascending=False)
)
df_exportacao_agrupado

,codigoTerminalCarregamento,conteineres
8,BRSSZ058,133732
5,BRSSZ016,95066
4,BRSSZ009,50161
7,BRSSZ057,2827
1,BRIOA001,15
2,BRPNG002,5
3,BRRIO002,2
6,BRSSZ031,2
0,BRIGI001,1


In [21]:
df_exportacao_agrupado_filtrado = (
    df_filtrado_agrupado.groupby('codigoTerminalCarregamento')['conteineres']
      .sum()
      .reset_index()
      .sort_values(by='conteineres', ascending=False)
)
df_exportacao_agrupado_filtrado

,codigoTerminalCarregamento,conteineres
5,BRSSZ058,86778
3,BRSSZ016,11612
2,BRSSZ009,4932
4,BRSSZ057,2532
1,BRPNG002,4
0,BRIGI001,1


In [22]:
sql_mercante_importacao_agrupado = '''select m.codigoTerminalDescarregamento, c.portoOrigemCarga,
m.portoCarregamento, count(i.codigoConteiner) as conteineres
from conhecimentosresumo c
inner join manifestosresumo m on c.manifestoCE = m.numero
inner join itensresumo i on i.numeroCEmercante = c.numeroCEmercante
where c.dataEmissao between "2025-06-01" and "2025-10-01"
and c.tipoTrafego = 5 and i.codigoConteiner !="" and m.portoDescarregamento = "BRSSZ"
group by m.codigoTerminalDescarregamento, c.portoOrigemCarga, m.portoCarregamento'''
df_mercante_importacao_agrupado = pd.read_sql(sql_mercante_importacao_agrupado, engine)

In [23]:
df_mercante_importacao_agrupado.rename(columns={'count(i.codigoConteiner)': 'conteineres'}, inplace=True)
df_mercante_importacao_agrupado

,codigoTerminalDescarregamento,portoOrigemCarga,portoCarregamento,conteineres
0,BRSSZ001,SIKOP,BRPNG,1
1,BRSSZ009,AEDXB,SGSIN,1
2,BRSSZ009,AEJEA,CNNGB,9
3,BRSSZ009,AEJEA,CNSHA,9
4,BRSSZ009,AEJEA,SGSIN,5
...,...,...,...,...
2068,BRSSZ058,ZADUR,GBLGP,1
2069,BRSSZ058,ZADUR,PTSIE,4
2070,BRSSZ058,ZAPLZ,PTSIE,8
2071,BRSSZ058,ZAZBA,ESLPA,1


In [24]:
df_mercante_importacao_agrupado.groupby('codigoTerminalDescarregamento').conteineres.sum().sort_values(ascending=False)

codigoTerminalDescarregamento
BRSSZ016    160364
BRSSZ058    145492
BRSSZ009     80707
BRSSZ057      1229
BRSSZ031       138
BRSSZ011        11
BRSSZ001         1
Name: conteineres, dtype: int64

In [25]:
df_importacao_agrupado = (
    df_mercante_importacao_agrupado.groupby('codigoTerminalDescarregamento')['conteineres']
      .sum()
      .reset_index()
      .sort_values(by='conteineres', ascending=False)
)
df_importacao_agrupado

,codigoTerminalDescarregamento,conteineres
3,BRSSZ016,160364
6,BRSSZ058,145492
1,BRSSZ009,80707
5,BRSSZ057,1229
4,BRSSZ031,138
2,BRSSZ011,11
0,BRSSZ001,1


In [26]:
df_mercante_importacao_agrupado['paisOrigem'] = df_mercante_importacao_agrupado['portoOrigemCarga'].str[:2]
ordenado_pais_impo = (
    df_mercante_importacao_agrupado.groupby('paisOrigem')['conteineres']
      .sum()
      .reset_index()
      .sort_values(by='conteineres', ascending=False)
)
ordenado_pais_impo[:10]

,paisOrigem,conteineres
18,CN,171619
105,US,37929
7,BE,20127
23,DE,19597
47,IN,12641
69,MX,11266
50,IT,10688
56,KR,8861
30,ES,8600
98,TH,7404


In [27]:
sql_fichas_expo = '''SELECT r.nome, count(o.id) as fichas FROM ovr_ovrs o
inner join ovr_recintos r on r.id = o.recinto_id
where o.tipooperacao = 2
and o.datahora between  "2025-06-01" and "2025-10-01"
group by r.nome'''
df_fichas_expo = pd.read_sql(sql_fichas_expo, engine)

In [28]:
df_fichas_expo

,nome,fichas
0,Brasil Terminal Portuario,152
1,ECOPORTO SANTOS - OPERADOR TECONDI,14
2,EMBRAPORT - EMPRESA BRASILEIRA DE TERMINAIS PO...,11
3,MARIMEX IPA,1
4,SANTOS BRASIL LOGISTICA S/A - CLIA GUARUJA,1
5,SANTOS BRASIL PARTICIPACOES S/A - OPERADOR,34


In [29]:
nome_para_sigla = {
    'Brasil': 'BTP',
    'SANTOS': 'SBT',
    'EMBRAPORT': 'DPW',
    'ECOPORTO': 'ECO'
}
codigo_para_sigla = {
    'BRSSZ058': 'BTP',
    'BRSSZ016': 'SBT',
    'BRSSZ009': 'DPW',
    'BRSSZ057': 'ECO'
}


df_fichas_expo.nome[1].split()[0]

'ECOPORTO'

In [30]:
df_fichas_expo['sigla'] = df_fichas_expo.nome.apply(lambda x: nome_para_sigla.get(x.split()[0]))

In [31]:
df_exportacao_agrupado['sigla'] =  df_exportacao_agrupado.codigoTerminalCarregamento.apply(lambda x: codigo_para_sigla.get(x))
df_exportacao_agrupado_filtrado['sigla'] =  df_exportacao_agrupado_filtrado.codigoTerminalCarregamento.apply(lambda x: codigo_para_sigla.get(x))

In [32]:
df_exportacao_fichas = pd.merge(df_exportacao_agrupado[df_exportacao_agrupado.sigla.notnull()], df_fichas_expo,
                                on='sigla', how='left')[['sigla', 'conteineres', 'fichas']]
df_exportacao_fichas

,sigla,conteineres,fichas
0,BTP,133732,152
1,SBT,95066,1
2,SBT,95066,34
3,DPW,50161,11
4,ECO,2827,14


In [33]:
df_exportacao_selecao_fichas = pd.merge(df_exportacao_fichas, df_exportacao_agrupado_filtrado[['sigla', 'conteineres']],
                                        on='sigla', suffixes=('_total', '_risco'))
df_exportacao_selecao_fichas = df_exportacao_selecao_fichas[['sigla', 'conteineres_total', 'conteineres_risco', 
                                                             'fichas']][df_exportacao_selecao_fichas['fichas'] > 2]

In [34]:
# Lista das colunas numéricas para calcular percentage
colunas = ['conteineres_total', 'conteineres_risco', 'fichas']

for i, coluna in enumerate(colunas):
    total = df_exportacao_selecao_fichas[coluna].sum()
    percent_col = f'% {i}'   # nome da nova coluna percentual
    df_exportacao_selecao_fichas[percent_col] = (df_exportacao_selecao_fichas[coluna] / total * 100).round(2)
    # Mover a nova coluna para logo após a original
    col_idx = df_exportacao_selecao_fichas.columns.get_loc(coluna)  # índice da coluna original
    col_pct_idx = df_exportacao_selecao_fichas.columns.get_loc(percent_col)  # índice da coluna percentual
    # Rearranjar colunas
    cols = list(df_exportacao_selecao_fichas.columns)
    cols.insert(col_idx + 1, cols.pop(col_pct_idx))  # coloca coluna percentual logo depois da original
    df_exportacao_selecao_fichas = df_exportacao_selecao_fichas[cols]

df_exportacao_selecao_fichas


,sigla,conteineres_total,% 0,conteineres_risco,% 1,fichas,% 2
0,BTP,133732,47.46,86778,81.98,152,72.04
2,SBT,95066,33.74,11612,10.97,34,16.11
3,DPW,50161,17.80,4932,4.66,11,5.21
4,ECO,2827,1.00,2532,2.39,14,6.64


In [35]:
totais = df_exportacao_selecao_fichas.select_dtypes(include='number').sum().astype(int)
linha_total = pd.DataFrame(totais).T  # transpose para linha
linha_total['sigla'] = 'Total'        # definir nome para a linha total
# Concatenar a linha de totais ao DataFrame original
df_exportacao_selecao_fichas = pd.concat([df_exportacao_selecao_fichas, linha_total], ignore_index=True)
df_exportacao_selecao_fichas

,sigla,conteineres_total,% 0,conteineres_risco,% 1,fichas,% 2
0,BTP,133732,47.46,86778,81.98,152,72.04
1,SBT,95066,33.74,11612,10.97,34,16.11
2,DPW,50161,17.80,4932,4.66,11,5.21
3,ECO,2827,1.00,2532,2.39,14,6.64
4,Total,281786,100.00,105854,100.00,211,100.00


In [36]:
df_exportacao_selecao_fichas['risco - total'] = df_exportacao_selecao_fichas.conteineres_total - df_exportacao_selecao_fichas.conteineres_risco
df_exportacao_selecao_fichas['risco / total'] = (df_exportacao_selecao_fichas.conteineres_risco / df_exportacao_selecao_fichas.conteineres_total * 100).round(2)
df_exportacao_selecao_fichas

,sigla,conteineres_total,% 0,conteineres_risco,% 1,fichas,% 2,risco - total,risco / total
0,BTP,133732,47.46,86778,81.98,152,72.04,46954,64.89
1,SBT,95066,33.74,11612,10.97,34,16.11,83454,12.21
2,DPW,50161,17.80,4932,4.66,11,5.21,45229,9.83
3,ECO,2827,1.00,2532,2.39,14,6.64,295,89.56
4,Total,281786,100.00,105854,100.00,211,100.00,175932,37.57


In [ ]:
df_importacao_agrupado['sigla'] = df_importacao_agrupado.codigoTerminalDescarregamento.apply(lambda x: codigo_para_sigla.get(x))

In [73]:
linha_total = df_importacao_agrupado['conteineres'].sum()
linha_total_df = pd.DataFrame({'sigla': ['Total'], 'conteineres': [linha_total]})
df_impo_resumo = pd.concat([df_importacao_agrupado[['sigla', 'conteineres']], linha_total_df]) 

In [79]:
df_impo_expo = pd.merge(df_exportacao_selecao_fichas, df_impo_resumo[['sigla', 'conteineres']], on='sigla')
df_impo_expo.rename(columns={'conteineres': 'escaneamento_impo'}, inplace=True)
df_impo_expo['escaneamento_total'] = df_impo_expo['conteineres_risco'] + df_impo_expo['escaneamento_impo']
df_impo_expo['total_geral'] = df_impo_expo['conteineres_total'] + df_impo_expo['escaneamento_impo']
df_impo_expo['incremento'] = (df_impo_expo['risco - total'] / df_impo_expo['total_geral'] * 100).round(2)
df_impo_expo


,sigla,conteineres_total,% 0,conteineres_risco,% 1,fichas,% 2,risco - total,risco / total,escaneamento_impo,escaneamento_total,total_geral,incremento
0,BTP,133732,47.46,86778,81.98,152,72.04,46954,64.89,145492,232270,279224,16.82
1,SBT,95066,33.74,11612,10.97,34,16.11,83454,12.21,160364,171976,255430,32.67
2,DPW,50161,17.80,4932,4.66,11,5.21,45229,9.83,80707,85639,130868,34.56
3,ECO,2827,1.00,2532,2.39,14,6.64,295,89.56,1229,3761,4056,7.27
4,Total,281786,100.00,105854,100.00,211,100.00,175932,37.57,387942,493796,669728,26.27


In [86]:
print('''    Tabela de movimentações de contêineres, segundo Siscomex CARGA. Período 01/06/2025 a 01/10/2025.
Sigla: recinto/operador de embarque (expo) ou desembarque (impo)
conteineres_total: Contêineres de CEs de exportação no período
conteineres_risco: Contêineres de CEs de exportação no período para países de escaneamento obrigatório
fichas: Número de Fichas de exportação/número de cargas selecionadas no período
escaneamento_impo: Contêineres de CEs de importação no período (100% escaneamento)
escaneamento_total: Total de contêineres de importação e exportação que deveriam ter sido escaneados no período.
total_geral: Total de contêineres de importação e exportação no período.
Obs.: 
No período, foram manifestados 209.000 CTRs vazios no fluxo de importação e 204.408 CTRs vazios no fluxo de exportação.
E foram capturadas 536.113 imagens no BD do Ajna.
(A priori, o BD do Ajna teria capturado 42.500 imagens a mais do que é obrigatório escanear, mas há muitos escaneamentos de passagem e 
de vazio a estufar. Contando os 413.000 contêineres vazios manifestados na importação e exportação vazios, ao invés de 42.500 imagens a mais,
podemos ter até 300.000 imagens faltando.)

Note-se que há uma diferença entre contêineres exportados e importados no período de mais de 108.000 unidades (quase 1.000 por dia),
mas pode ser que esta diferença esteja compensada por 120.286 contêineres vazios do tipo BCE no período.
''')
df_impo_expo

    Tabela de movimentações de contêineres, segundo Siscomex CARGA. Período 01/06/2025 a 01/10/2025.
Sigla: recinto/operador de embarque (expo) ou desembarque (impo)
conteineres_total: Contêineres de CEs de exportação no período
conteineres_risco: Contêineres de CEs de exportação no período para países de escaneamento obrigatório
fichas: Número de Fichas de exportação/número de cargas selecionadas no período
escaneamento_impo: Contêineres de CEs de importação no período (100% escaneamento)
escaneamento_total: Total de contêineres de importação e exportação que deveriam ter sido escaneados no período.
total_geral: Total de contêineres de importação e exportação no período.
Obs.: 
No período, foram manifestados 209.000 CTRs vazios no fluxo de importação e 204.408 CTRs vazios no fluxo de exportação.
E foram capturadas 536.113 imagens no BD do Ajna.
(A priori, o BD do Ajna teria capturado 42.500 imagens a mais do que é obrigatório escanear, mas há muitos escaneamentos de passagem e 
de vaz

,sigla,conteineres_total,% 0,conteineres_risco,% 1,fichas,% 2,risco - total,risco / total,escaneamento_impo,escaneamento_total,total_geral,incremento
0,BTP,133732,47.46,86778,81.98,152,72.04,46954,64.89,145492,232270,279224,16.82
1,SBT,95066,33.74,11612,10.97,34,16.11,83454,12.21,160364,171976,255430,32.67
2,DPW,50161,17.80,4932,4.66,11,5.21,45229,9.83,80707,85639,130868,34.56
3,ECO,2827,1.00,2532,2.39,14,6.64,295,89.56,1229,3761,4056,7.27
4,Total,281786,100.00,105854,100.00,211,100.00,175932,37.57,387942,493796,669728,26.27


# Recuperar informações de REDEX e OPERADORES

In [39]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204')" # Redex da Localfrio/Movecta e Redex da Santos Brasil Clia
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [40]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento

In [41]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 22 toneladas ou mais, então, ele está carregado (não é vazio)

In [42]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 22 toneladas ou mais, então, ele está carregado (não é vazio)

In [43]:
# SQL para saida_redex (alias c)
sql_saida_redex = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, direcao
FROM apirecintos_acessosveiculo
WHERE
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'S'
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND numeroConteiner !=''
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento

In [44]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [45]:
df_entrada_operadores

,id,placa,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento
0,13082029,EWI0006,ACLU9671797,8931318,2025-10-13 15:50:32,C,E,49612377000190,27372040892,FERNANDO APARECIDO MARCIANO,35250912259649000100550010000383851961614243,
1,13081972,BWY2A55,GCNU1330838,8931318,2025-10-13 15:56:33,C,E,02077058000169,13357678861,GERALDO DOMINGUES DE OLIVEIRA,31250919818301000155550010001648451004399350,
2,13081969,CUA4J91,SEGU2932245,8931318,2025-10-13 15:57:01,C,E,02077058000169,27769375800,MARCIO JOSE CARDOSO DOS SANTOS,31250919818301000155550010001648451004399350,
3,13081927,HQG6301,HOYU2623280,8931318,2025-10-13 15:57:59,C,E,01896217000194,04112860803,JOSE FRANCISCO DA SILVA,,
4,13081973,EJW2C71,BMOU2157475,8931318,2025-10-13 15:58:57,C,E,12014047000193,26909352874,RUBENS LEAL DA SILVA,,
...,...,...,...,...,...,...,...,...,...,...,...,...
27516,13223665,NVQ5J82,HAMU2390392,8931404,2025-10-19 20:08:43,C,E,01896217000194,19230715808,UBIRATAN SCHEK,,
27517,13223824,FZT0H26,TLLU8274977,8931404,2025-10-19 20:23:57,C,E,58180316000192,29986271886,FLAVIO EDUARDO SARAIVA PEREIRA,,
27518,13223694,KMC9348,TRHU4927459,8931404,2025-10-19 21:09:49,C,E,21425093000176,44779261899,LEONARDO SILVA DOS ANJOS,,
27519,13223698,DBM7C46,CMAU2646606,8931404,2025-10-19 21:16:53,C,E,21425093000176,95413790449,JEFFERSON JOSE DA SILVA LIMA,,


In [46]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [47]:
df_saida_redex

,id,placa,numeroConteiner,codigoRecinto,dataHoraOcorrencia,direcao
0,13081110,EOX4H74,SZLU9712502,8931309,2025-10-13 16:12:05,S
1,13081120,FAA6J12,OOLU6432061,8931309,2025-10-13 16:37:57,S
2,13081122,BEZ4C32,ONEU9090470,8931309,2025-10-13 16:45:34,S
3,13081128,BTA3650,CGMU5120423,8931309,2025-10-13 17:15:42,S
4,13081217,DAJ4H74,FTAU1184406,8931309,2025-10-13 17:30:55,S
...,...,...,...,...,...,...
8331,13234683,FOZ9J84,SUDU7752020,8933204,2025-10-19 22:55:55,S
8332,13234684,RHZ6F94,HASU4981230,8933204,2025-10-19 23:00:56,S
8333,13234696,FEI9J41,CXDU2180047,8933204,2025-10-19 23:46:11,S
8334,13234697,FIR5718,NYKU3722921,8933204,2025-10-19 23:49:36,S


In [48]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placa', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [49]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [50]:
# df_saidas_entradas

In [51]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [52]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

Buscando ids no MongoDB:   0%|          | 0/6052 [00:00<?, ?it/s]

Registros: 6052, imagens: 2318


In [53]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'numeroConteiner': 'contêiner',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,id_S_REDEX,Redex,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,13082000,DVS1I68,GCNU1337468,8931318,2025-10-13 16:39:22,C,E,02077058000169,37415733886,JEFEERSON DE SOUZA FELIX RODRIGUES,35251053009825000133550070006843801026316983,,13100648,8932797,2025-10-13 15:48:48,S,0.842778,68ee36dccca6eeddad8174df
1,13082041,DPC9006,GCNU1306512,8931318,2025-10-13 16:43:03,C,E,02077058000169,30792294874,CLAYTON MACHADO DE CARVALHO,35251053009825000133550070006843801026316983,,13100656,8932797,2025-10-13 15:57:26,S,0.760278,68ee36dbcca6eeddad8174c7
2,13082005,DVS1I67,ACLU2787825,8931318,2025-10-13 16:44:59,C,E,02077058000169,03924861803,LUIZ FERREIRA BATISTA,35251053009825000133550070006843801026316983,,13100660,8932797,2025-10-13 16:05:22,S,0.660278,68ee36dacca6eeddad81749f
3,13082051,TJM0A83,SEGU3282268,8931318,2025-10-13 17:08:02,C,E,01984600000366,07000746866,ANTONIO FERREIRA DE MELO,31250901325285000101550010000066911110108114,,13098616,8932775,2025-10-13 16:17:41,S,0.839167,68ee3725cca6eeddad817cd9
4,13082086,DPB9G17,GCNU1363430,8931318,2025-10-13 18:32:14,C,E,02077058000169,43706665549,JOSE OSMAR DANTAS,31250919818301000155550010001654861004406328,,13100698,8932797,2025-10-13 17:59:07,S,0.551944,68ee374522ad104e0013f1c2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6047,13222176,FAF7C65,TCNU3762406,8931404,2025-10-18 10:09:29,C,E,04211559000111,88487911587,SÃ?RGIO ALVES BARRETO,,,13236636,8932747,2025-10-18 09:03:32,S,1.099167,None
6048,13222268,BBJ7225,UACU4090268,8931404,2025-10-18 10:59:33,C,E,06235812000100,05243684813,EDMUR OLIVEIRA DE ALBUQUERQUE,,,13239206,8932792,2025-10-18 09:54:21,S,1.086667,68f624930413a90d8c882cfb
6049,13222271,BWF7G52,HAMU1069599,8931404,2025-10-18 11:01:22,C,E,06235812000100,30219438889,ANDERSON RODRIGUES PEREIRA,,,13239207,8932792,2025-10-18 09:54:30,S,1.114444,68f6231d63c124656506e682
6050,13222280,MQA5041,NIDU2370350,8931404,2025-10-18 11:05:37,C,E,06235812000100,73009997868,VALDIR RODRIGUES PEREIRA,,,13239209,8932792,2025-10-18 09:54:38,S,1.183056,68f6245c0413a90d8c882737


In [54]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

,placa,contêiner,Redex,Saída REDEX,Operador,Entrada Operador,transitTime,_id,html
272,GFR6A86,EMCU5611334,8932788,2025-10-13 22:15:18,8931356,2025-10-13 20:35:35,-1.661944,None,
274,NYT4G45,PCIU6129253,8932788,2025-10-13 22:20:06,8931356,2025-10-13 20:38:41,-1.690278,None,
298,RXM0B18,TLLU1490260,8932747,2025-10-14 04:35:33,8931356,2025-10-13 23:18:45,-5.280000,None,
556,DVT6B08,TEMU9338329,8932761,2025-10-15 10:37:00,8931356,2025-10-14 17:08:09,-17.480833,None,
622,MSM7C38,TCNU7946361,8932754,2025-10-15 18:51:00,8931356,2025-10-15 10:07:17,-8.728611,68efd086223c139533d40db1,
1245,TIO0F69,HGTU9616880,8932775,2025-10-16 06:54:56,8931356,2025-10-16 12:05:55,5.183056,68f1348e48a0207876782d9e,
1391,EQU9I31,YMLU5517290,8932788,2025-10-17 15:03:41,8931356,2025-10-16 16:01:36,-23.034722,None,
1466,OZN4B18,YMLU5418430,8932773,2025-10-16 14:47:49,8931356,2025-10-16 19:12:49,4.416667,None,
1794,CUD3F89,MRKU3849119,8932796,2025-10-17 03:40:28,8931356,2025-10-17 07:58:58,4.308333,None,
1931,DBC2E26,TTNU1288538,8932778,2025-10-17 16:58:00,8931356,2025-10-17 12:24:32,-4.557778,None,


In [55]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [56]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

In [57]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placa', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 3) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placa', 'numeroConteiner'], right_on=['placa', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 3) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) &
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

SyntaxError: invalid syntax (1004912324.py, line 26)